# Data mining of statistics from OECD on the countries of USA and Spain

**Source: https://data.oecd.org/**

In [300]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib3
import ssl
import re

**In case that the server doesn't support "RFC 5746 secure renegotiation" and the client is using OpenSSL 3, one needs to:**

Source: https://stackoverflow.com/questions/71603314/ssl-error-unsafe-legacy-renegotiation-disabled

In [301]:
class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [302]:
url_usa = "https://data.oecd.org/united-states.htm"
url_spain = "https://data.oecd.org/spain.htm"

In [303]:
html_data_usa = get_legacy_session().get(url_usa).text
html_data_spain  = get_legacy_session().get(url_spain).text

In [304]:
soup_usa = BeautifulSoup(html_data_usa, 'html')
soup_spain = BeautifulSoup(html_data_spain, 'html')

In [400]:
read_html_pandas_data_usa = pd.read_html(str(soup_usa))
read_html_pandas_data_spain = pd.read_html(str(soup_spain))

In [69]:
table0usa = read_html_pandas_data_usa[1]

In [70]:
table0usa

,Development,Development.1,Latest,Trend,Ranking
0,Distribution of net ODAIndicator,NaN,11 808.8 Least developed countries Million US ...,Least developed countries Million US dollars 2...,Least developed countries Million US dollars 2...
1,Grants by private agencies and NGOsIndicator,NaN,4 953.0 Total Million US dollars 2021 United S...,Total Million US dollars 2002-2021 United Stat...,NaN
2,Net ODAIndicator,NaN,0.2 ODA grant equivalent % of gross national i...,ODA grant equivalent % of gross national incom...,ODA grant equivalent % of gross national incom...
3,Private flowsIndicator,NaN,159 212.2 Total Million US dollars 2021 United...,Total Million US dollars 2002-2021 United Stat...,Total Million US dollars 2020 United States (r...
4,Total official and private flowsIndicator,NaN,229 521.9 Total Million US dollars 2021 United...,Total Million US dollars 2002-2021 United Stat...,Total Million US dollars 2021 United States (r...


In [79]:
table0usa.columns.values.tolist()[0]

'Development'

In [71]:
newtable = table0usa.drop(table0usa.columns[[1,3,4]], axis=1)
newtable

,Development,Latest
0,Distribution of net ODAIndicator,11 808.8 Least developed countries Million US ...
1,Grants by private agencies and NGOsIndicator,4 953.0 Total Million US dollars 2021 United S...
2,Net ODAIndicator,0.2 ODA grant equivalent % of gross national i...
3,Private flowsIndicator,159 212.2 Total Million US dollars 2021 United...
4,Total official and private flowsIndicator,229 521.9 Total Million US dollars 2021 United...


In [72]:
newtable.rename(columns = { newtable.columns[1]: "Indicator for USA" }, inplace = True)
newtable

,Development,Indicator for USA
0,Distribution of net ODAIndicator,11 808.8 Least developed countries Million US ...
1,Grants by private agencies and NGOsIndicator,4 953.0 Total Million US dollars 2021 United S...
2,Net ODAIndicator,0.2 ODA grant equivalent % of gross national i...
3,Private flowsIndicator,159 212.2 Total Million US dollars 2021 United...
4,Total official and private flowsIndicator,229 521.9 Total Million US dollars 2021 United...


In [77]:
# newtable["Revenue"] = newtable['Revenue'].str.replace(',|\$','', regex = True)
newtable["Development"] = newtable["Development"].str.replace(r'Indicator', '', regex = True)
newtable["Indicator for USA"] = newtable["Indicator for USA"].str.replace(r'United States', '', regex = True)
newtable

,Development,Indicator for USA
0,Distribution of net ODA,11 808.8 Least developed countries Million US ...
1,Grants by private agencies and NGOs,4 953.0 Total Million US dollars 2021 Million...
2,Net ODA,0.2 ODA grant equivalent % of gross national i...
3,Private flows,159 212.2 Total Million US dollars 2021 Milli...
4,Total official and private flows,229 521.9 Total Million US dollars 2021 Milli...


In [87]:
def data_pipeline(raw_table, country = str):
    title1 = raw_table.columns.values.tolist()[0]
    title2 = "Indicator for {0}".format(country)
    newtable = raw_table.drop(raw_table.columns[[1,3,4]], axis=1)
    newtable.rename(columns = { newtable.columns[1]: "{0}".format(title2) }, inplace = True)
    
    newtable[title1] = newtable[title1].str.replace(r'Indicator', '', regex = True)
    newtable[title2] = newtable[title2].str.replace(r'United States', '', regex = True)
    newtable[title2] = newtable[title2].str.replace(r'Spain', '', regex = True)
    
    return newtable

In [99]:
# country = "USA"
# raw_table = read_html_pandas_data_usa[9]

# country = "Spain"
# raw_table = read_html_pandas_data_spain[9]

# table = data_pipeline(raw_table, country)
# table

,Innovation and Technology,Indicator for Spain
0,Gross domestic spending on R&D,1.4 Total % of GDP 2021 % of GDP
1,ICT value added,4.6 Total % of value added 2011 % of value added
2,Internet access,96.1 Total % of all households 2022 % of all ...
3,Mobile broadband subscriptions,109.9 Total Per 100 inhabitants Q2-2022 Per 1...
4,Triadic patent families,321.9 Total Number 2020 Number


In [127]:
def table_unificator(table1, table2):

    new_table = table1.drop(table1.columns[[0]], axis=1)
    table2 = table2.join(new_table)

    return table2

def get_nth_table(n = int):
    table_x = data_pipeline(read_html_pandas_data_usa[n], "USA")
    table_y = data_pipeline(read_html_pandas_data_spain[n], 'Spain')

    table_z = table_unificator(table_x, table_y)
    
    return table_z


In [126]:
table_x = data_pipeline(read_html_pandas_data_usa[7], "USA")
table_y = data_pipeline(read_html_pandas_data_spain[7], 'Spain')

table_z = table_unificator(table_x, table_y)
table_z

,Government,Indicator for Spain,Indicator for USA
0,General government spending,50.6 Total % of GDP 2021 % of GDP,44.9 Total % of GDP 2021 % of GDP
1,Government reserves,27 576.4 Total SDR millions Q4-2014 SDR millions,91 322.5 Total SDR millions Q4-2014 SDR millions
2,Tax on corporate profits,2.7 Total % of GDP 2021 % of GDP,1.6 Total % of GDP 2021 % of GDP
3,Tax revenue,38.4 Total % of GDP 2021 % of GDP,26.6 Total % of GDP 2021 % of GDP
4,Tax wedge,39.3 Total % of labour cost 2021 % of labour ...,28.4 Total % of labour cost 2021 % of labour ...


In [145]:
table_7 = get_nth_table(n=3)
table_7

,Education,Indicator for Spain,Indicator for USA
0,Education spending,14 237 Tertiary US dollars/student 2019 US do...,35 347 Tertiary US dollars/student 2019 US do...
1,Mathematics performance (PISA),485 Boys Mean score 2018 Mean score,482 Boys Mean score 2018 Mean score
2,Reading performance (PISA),485 Boys Mean score 2015 Mean score,494 Boys Mean score 2018 Mean score
3,Science performance (PISA),484 Boys Mean score 2018 Mean score,503 Boys Mean score 2018 Mean score
4,"Youth not in employment, education or training...",8.5 15-19 year-old men % in same age group 202...,9.3 15-19 year-old men % in same age group 202...


###  In case one needs manipulation of strings with regular expression

In [179]:
### string.split() ###

In [204]:
string = table_7.iloc[0,1]
string

'14 237 Tertiary US dollars/student 2019  US dollars/student'

In [181]:
# expression = '.+?(?=[a-zA-Z])'
# all_strings = re.search(expression, string)
# the_number = all_strings[0]
# the_number